# PROMPT RESULT ANALYSIS

In [ ]:
import pickle
import os
import json
from eval import choose_plan

pred_plans = pickle.load(open('result/alfred/prompt10/vanilla/valid_seen_2.p'))

not_executable = []
correct = []

for goal in pred_plans:
    best_plan = choose_plan(pred_plans[goal]['plan'])
    for ep in os.listdir('data/alfred_data/json_2.1.0/valid_seen'):
        for trial in os.listdir(os.path.join('data/alfred_data/json_2.1.0/valid_seen', ep)):
            traj_data = json.load(open(os.path.join('data/alfred_data/json_2.1.0/valid_seen', ep, trial, 'traj_data.json')))
            anns = [ann['task_desc'] for ann in traj_data['turk_annotations']['anns']]
            if goal not in anns:
                continue
            task_type = traj_data['task_type']
            pddl_param = traj_data['pddl_params']
            if best_plan is None:
                not_executable.append({
                    'goal': goal,
                    'task_type': task_type,
                    'pddl_param':pddl_param
                    })
            


# HuggingFace 써보기

## pipeline

자동으로 기본 모델과 preprocessing을 사용해서 inference한다.

In [ ]:
from transformers import pipeline

generator = pipeline() 
# task='', model='',  -- huggingface hub에 있는대로 가져와야 함. ex) openai/whisper-large
# task specific parameters는 task API 확인. ex) transformers.[taskname]Pipeline.call()의 파라미터
result = generator(input_data) # 여러개면 list of data

In [2]:
from transformers import pipeline

generator = pipeline(model='gpt2', device=0)
print(generator('New way to solve the problem is'))

/home/user/anaconda3/envs/language-planner/lib/python3.7/site-packages/transformers/generation/utils.py:1212: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  "You have modified the pretrained model configuration to control generation. This is a"
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/user/anaconda3/envs/language-planner/lib/python3.7/site-packages/transformers/generation/utils.py:1302: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


[{'generated_text': 'New way to solve the problem is to give your children a new school that will be similar to yours. The first step is to set up a new school that will teach English and you must send him to a similar school in a different country. The'}]


## 지정해서 사용하기
pipeline = tokenizer + model

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained() # 모델 이름
encoded_input = tokenizer('') # input, padding, truncation, return_tensors='pt', ...

decoding method
* greedy search (default)
* contrastive search - non-repetitive, long
* beam-search decoding - 전체 sequence를 참조해서 가장 prob 높은 것을 고른다, num beams는 keep하는 hypothesis의 수

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = 'gpt2-xl' # gpt2-xl
tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer('Correct this to standard English:\n slice WineBottle\n', return_tensors='pt')
model = AutoModelForCausalLM.from_pretrained(model_name)
outputs = model.generate(**inputs, num_beams=5, max_new_tokens=50)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Correct this to standard English:\n slice WineBottle\nThis will give you a list of all the WineBottles you have in your inventory. If you have more than one WineBottle in your inventory, you will get a list of all the WineBottles you have in your inventory. If you']

# openai api

앗... 여러개 할 수 있었음

In [2]:
import openai
from utils import config

openai.api_key = config.OPENAI['api_key']
openai.organization = config.OPENAI['organization']

prompt = ["Correct this to standard English:\n\n%s\n"%('abs'), "Correct this to standard English:\n\n%s\n"%('do something'), "Correct this to standard English:\n\n%s\n"%('can you help me out?')]

response = openai.Completion.create(
                model = 'text-davinci-003',
                prompt = prompt,
                temperature = 0,
                logprobs=1,
                n = 1,
                max_tokens = 60,
                stop = None
            )


In [5]:
print([c.text for c in response.choices])

['\nAbs', '\nDo something.', '\nCan you help me?']


In [3]:
openai.Model.list()

<OpenAIObject list at 0x7f41adecbcb0> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sa

# Memo

In [14]:
import json
import pprint

threas = 1/100
pp = pprint.PrettyPrinter()
confusion = json.load(open('confusion_matrix_valid_seen.json', 'r'))
for key, graph in confusion.items():
    for _from in graph:
        for _to in graph[_from]:
            freq = graph[_from][_to]
            if freq > threas*820:
                print(f"{key}: {_from} -> {_to}")

total: Knife -> Apple
total: Knife -> Tomato
total: Knife -> Lettuce
total: SideTable -> DiningTable
total: ButterKnife -> Tomato
total: ButterKnife -> Apple
total: Spoon -> Ladle
total: Cup -> Mug
object_target: Knife -> Apple
object_target: Knife -> Tomato
object_target: Knife -> Lettuce
object_target: ButterKnife -> Tomato
object_target: ButterKnife -> Apple
object_target: Spoon -> Ladle
mrecep_target: Cup -> Mug
parent_target: SideTable -> DiningTable


In [3]:
import pickle
import pprint
pp = pprint.PrettyPrinter()

pred_seen = pickle.load(open('result/alfred/prompt10/vanilla/valid_seen_6.p', 'rb'))
for goal in pred_seen:
    if goal == 'Turning on a lamp':
        pp.pprint(pred_seen[goal])

{'plan': [[['PickupObject', 'Book', 'SideTable'],
           ['ToggleObject', 'FloorLamp', 'FloorLamp']],
          [['PickupObject', 'Book', 'Shelf'],
           ['ToggleObject', 'DeskLamp', 'DeskLamp']]],
 'prompt_logp': [-0.922696, -2.116732],
 'trans_cossim': [array([2.0000002, 1.9999999], dtype=float32),
                  array([1.       , 1.0000001], dtype=float32)],
 'vote': [2, 1]}


In [ ]:
import json
import pprint
pp = pprint.PrettyPrinter()

action1 = json.load(open('data/available_actions1.json', 'r'))
action2 = json.load(open('data/available_actions2.json', 'r'))

for sentence1, triplet in action1.items():
    sentence2 = list(action2.keys())[list(action2.values()).index(triplet)]
    if sentence1.lower() != sentence2.lower():
        pp.pprint(triplet)        
        print(f"{sentence1}\n{sentence2}")
        print('-'*30+'\n')

In [9]:
import json
import random
import pprint
pp = pprint.PrettyPrinter()

gt_unseen = json.load(open('data/triplet/val_unseen_noappended.json', 'r'))

selected_keys = list(random.sample(gt_unseen.keys(), 2))
selected_values = [ list(gt_unseen.values())[list(gt_unseen.keys()).index(k) ]for k in selected_keys]
pp.pprint(dict(zip(selected_keys, selected_values)))

{'Put a chilled potato into the microwave.': [[['PickupObject',
                                                'Potato',
                                                'SinkBasin'],
                                               ['CoolObject',
                                                'Potato',
                                                'Fridge'],
                                               ['PutObject',
                                                'Potato',
                                                'Microwave']]],
 'take a coffee mug from a book shelf and place it on a desk': [[['PickupObject',
                                                                  'Mug',
                                                                  'Shelf'],
                                                                 ['PutObject',
                                                                  'Mug',
                                                                  'Desk

In [3]:
import pickle

data = pickle.load(open('result/alfred/prompt4/vanilla/valid_seen_0.p', 'rb'))
print(data)

{'Turn the green lamp on while carrying a tennis racquet.': {'plan': [[['PickupObject', 'TennisRacket', ''], ['CoolObject', 'FloorLamp', 'Fridge'], ['ToggleObject', 'LightSwitch', 'LightSwitch']]], 'vote': [3], 'prompt_logp': [-0.86729753], 'trans_cossim': [array([3.       , 1.5541799, 2.0124967], dtype=float32)]}, 'Carry a tennis racket while turning on a lamp': {'plan': [[['PickupObject', 'TennisRacket', ''], ['SliceObject', 'FloorLamp', 'Box'], ['PutObject', 'LightSwitch', 'Box'], ['ToggleObject', 'LightSwitch', 'LightSwitch']], [['PickupObject', 'TennisRacket', ''], ['ToggleObject', 'DeskLamp', 'DeskLamp'], ['CoolObject', 'TennisRacket', 'Fridge'], ['', '', '']], [['PickupObject', 'TennisRacket', ''], ['PutObject', 'LightSwitch', 'Bed'], ['ToggleObject', 'LightSwitch', 'LightSwitch'], ['', '', '']]], 'vote': [1, 1, 1], 'prompt_logp': [-16.22614, -25.621986, -9.938242], 'trans_cossim': [array([1.        , 0.62471   , 0.7017941 , 0.84170985], dtype=float32), array([ 1.       ,  1.000

In [1]:
import json
import os
from utils.plan_module import Plan
from tqdm.notebook import tqdm

pred = json.load(open('result/alfred/roberta/valid_seen-sentence@4-1.json', 'r'))
root = 'data/alfred_data/json_2.1.0/valid_seen'
plan2triplet = json.load(open('data/available_actions1.json', 'r'))

err_cnt, suc_cnt, cnt = 0, 0, 0
for goal in tqdm(pred):
    pred_str_plan = pred[goal]['similar']['plan'][-1][0]
    pred_plan = [plan2triplet[str_action] for str_action in pred_str_plan]
    pred_plan = Plan(triplets=pred_plan)
    
    found = False
    for ep in os.listdir(root):
        for trial in os.listdir(os.path.join(root, ep)):
            traj_data = json.load(open(os.path.join(root, ep, trial, 'traj_data.json'), 'r'))
            anns = [ann['task_desc'] for ann in traj_data['turk_annotations']['anns']]
            if goal in anns:
                found = True
                task_type = traj_data['task_type']
                pddl_params = traj_data['pddl_params']
                break
        if found:
            break
    
    executable = pred_plan.is_executable()
    suc = pred_plan.is_plan_fulfilled(task_type, pddl_params)
    
    cnt += 1
    if suc:
        suc_cnt += 1
    if not executable:
        err_cnt += 1
        
print(f"{suc_cnt}/{cnt} : {(suc_cnt/cnt)*100:.2f}")
print(f"{err_cnt}/{cnt} : {(err_cnt/cnt)*100:.2f}")

  0%|          | 0/807 [00:00<?, ?it/s]

606/807 : 75.09
0/807 : 0.00


In [88]:
print(f"{suc_cnt}/{cnt} : {(suc_cnt/cnt)*100:.2f}")
print(f"{err_cnt}/{cnt} : {(err_cnt/cnt)*100:.2f}")

560/759 : 73.78
0/759 : 0.00


In [4]:
import json
import os
from utils.plan_module import Plan
from tqdm.notebook import tqdm
import pprint

pp = pprint.PrettyPrinter()

pred = json.load(open('data/triplet/val_unseen_noappended.json', 'r'))
root = 'data/alfred_data/json_2.1.0/valid_unseen'
plan2triplet = json.load(open('data/available_actions1.json', 'r'))

err_cnt, suc_cnt, cnt = 0, 0, 0
for goal in tqdm(pred):
    # pred_str_plan = pred[goal]['similar']['plan'][-1][0]
    # pred_plan = [plan2triplet[str_action] for str_action in pred_str_plan]
    pred_plan = pred[goal][0]
    pred_plan = Plan(triplets=pred_plan)
    
    found = False
    for ep in os.listdir(root):
        for trial in os.listdir(os.path.join(root, ep)):
            traj_data = json.load(open(os.path.join(root, ep, trial, 'traj_data.json'), 'r'))
            anns = [ann['task_desc'] for ann in traj_data['turk_annotations']['anns']]
            if goal in anns:
                found = True
                task_type = traj_data['task_type']
                pddl_params = traj_data['pddl_params']
                break
        if found:
            break
    
    executable = pred_plan.is_executable(verbose=True)
    suc = pred_plan.is_plan_fulfilled(task_type, pddl_params)
    
    cnt += 1
    if suc:
        suc_cnt += 1
    else:
        print(goal)
        pp.pprint(str(pred_plan))
        pp.pprint(task_type)
        pp.pprint(pddl_params)
        pp.pprint(os.path.join(ep, trial))
    if not executable:
        err_cnt += 1
        
print(f"{suc_cnt}/{cnt} : {(suc_cnt/cnt)*100:.2f}")
print(f"{err_cnt}/{cnt} : {(err_cnt/cnt)*100:.2f}")

  0%|          | 0/759 [00:00<?, ?it/s]

Turn on the desk lamp.
'1. Pickup   CD GarbageCan\n2. Toggle   DeskLamp DeskLamp\n'
'look_at_obj_in_light'
{'mrecep_target': '',
 'object_sliced': False,
 'object_target': 'Mug',
 'parent_target': '',
 'toggle_target': 'DeskLamp'}
'look_at_obj_in_light-Mug-None-DeskLamp-308/trial_T20190908_201421_021646'
Turn on the living room lamp. 
'1. Pickup   CellPhone ArmChair\n2. Toggle   FloorLamp FloorLamp\n'
'look_at_obj_in_light'
{'mrecep_target': '',
 'object_sliced': False,
 'object_target': 'RemoteControl',
 'parent_target': '',
 'toggle_target': 'FloorLamp'}
'look_at_obj_in_light-RemoteControl-None-FloorLamp-219/trial_T20190909_032758_971610'
757/759 : 99.74
0/759 : 0.00


In [17]:
import pickle
import pprint

pp = pprint.PrettyPrinter()

pred_seen = pickle.load(open('result/alfred/prompt4/vanilla/valid_seen_0.p', 'rb'))

candidate = [
    'Carry a tennis racket while turning on a lamp',
]

for goal in candidate:
    c_idx = [idx for idx, g in enumerate(pred_seen.keys()) if g.lower().strip() == goal.lower().strip()]
    pp.pprint(list(pred_seen.values())[c_idx[0]])
    print()

{'plan': [[['PickupObject', 'TennisRacket', ''],
           ['SliceObject', 'FloorLamp', 'Box'],
           ['PutObject', 'LightSwitch', 'Box'],
           ['ToggleObject', 'LightSwitch', 'LightSwitch']],
          [['PickupObject', 'TennisRacket', ''],
           ['ToggleObject', 'DeskLamp', 'DeskLamp'],
           ['CoolObject', 'TennisRacket', 'Fridge'],
           ['', '', '']],
          [['PickupObject', 'TennisRacket', ''],
           ['PutObject', 'LightSwitch', 'Bed'],
           ['ToggleObject', 'LightSwitch', 'LightSwitch'],
           ['', '', '']]],
 'prompt_logp': [-16.22614, -25.621986, -9.938242],
 'trans_cossim': [array([1.        , 0.62471   , 0.7017941 , 0.84170985], dtype=float32),
                  array([ 1.       ,  1.0000001,  0.5574468, -1.       ], dtype=float32),
                  array([ 1.       ,  0.6846826,  0.9187366, -1.       ], dtype=float32)],
 'vote': [1, 1, 1]}



In [5]:
from utils.plan_module import Plan
import pickle
import pprint
import numpy as np

pp = pprint.PrettyPrinter()
# eval할 때 쓴 plan을 드려야 하네..? ^^
pred_seen = pickle.load(open('result/alfred/prompt20/vanilla/valid_unseen_2.p', 'rb'))

for goal in pred_seen:
    if 'slice' not in goal.lower():
        continue
    plans = pred_seen[goal]['plan']

    print(goal)
    try:
        test_plan = Plan(triplets=plans[0])
    except:
        continue
    pp.pprint(test_plan.high_actions)
    print()
    pp.pprint(test_plan.get_low_actions())
    print('\n=============\n')

In [1]:
import pickle
import random
import pprint
pp = pprint.PrettyPrinter()

data = pickle.load(open('result/prompt10/vanilla/valid_seen.p', 'rb'))

sampled_keys = random.sample(list(data.keys()), 3)
sample_dict = dict(zip(sampled_keys, [data[k] for k in sampled_keys]))

pp.pprint(sample_dict)

{'Hold a knife in front of a closed fridge door.': OrderedDict([('root',
                                                                 'data/alfred_data/json_2.1.0/valid_seen/pick_two_obj_and_place-LettuceSliced-None-Fridge-1/trial_T20190906_181501_970690/traj_data.json'),
                                                                ('low_actions',
                                                                 ['GotoLocation',
                                                                  'PickupObject',
                                                                  'GotoLocation',
                                                                  'OpenObject',
                                                                  'PutObject',
                                                                  'CloseObject']),
                                                                ('low_classes',
                                                                 ['ButterKn